In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import xgboost as xgb

import time
print('sleeping')
time.sleep(1800)
print('sleep done =======================')

# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../features/*.pkl')):
    if '3_feat' in feat:
        continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

sleeping
sleep done =======================
file path ../features/glove_cnn2d_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_gru_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cnn_v2_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_cudnn_gru_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_gru_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/glove_lstm_v1_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/lr_feat1.pkl
(159571, 6) (153164, 6)
file path ../features/lr_feat2.pkl
(159571, 6) (153164, 6)
file path ../features/lstm_attention_fasttext_10_feat.pkl
(159571, 6) (153164, 6)
file path ../features/lstm_attention_fasttext_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/lstm_attention_glove_4_feat.pkl
(159571, 6) (153164, 6)
file path ../features/mnb_feat1.pkl
(159571, 6) (153164, 6)
file path ../features/mnb_feat2.pkl
(159571

In [2]:
from sklearn.model_selection import KFold
def simple_ens(model_name,k=3,rnd=233):
    kf = KFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    cache_test_pred = np.zeros((153164,6))
    single_best = 100
    single_best_pred = None
    all_train_loss_l,all_val_loss_l = 0,0
    
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        val_loss_l,train_loss_l = 0,0
        d_test = xgb.DMatrix(test_x)
        
        # share params
        params = {
                'subsample': 0.9,
                'eta': 0.05,
                'max_depth': 3,
                'eval_metric':'logloss',
                #'eval_metric':'auc',
                'objective':'binary:logistic',
                'scale_pos_weight':0.9,
                'colsample_bylevel':0.9,
                'colsample_bytree':0.9,
            
                }
        
        # train for each class
        for i in range(6):
            d_train = xgb.DMatrix(curr_x, curr_y[:,i])
            d_valid = xgb.DMatrix(hold_out_x, hold_out_y[:,i])
            watchlist = [(d_train, 'train'), (d_valid, 'valid')]

            model = xgb.train(params, d_train, 1000, watchlist,
                              early_stopping_rounds=50,
                              verbose_eval=None)
            print(i)
            try:
                curr_train_loss = log_loss(curr_y[:,i],model.predict(d_train))
                curr_val_loss = log_loss(hold_out_y[:,i],model.predict(d_valid))
                print(curr_train_loss,curr_val_loss)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
            except:
                pass
            curr_test_pred = model.predict(d_test)
            
            test_pred[:,i] += curr_test_pred
            cache_test_pred[:,i] += curr_test_pred
            
        # avg 6 class
        train_loss_l = train_loss_l/6
        val_loss_l = val_loss_l/6
        print('this fold avg train',train_loss_l,'avg val',val_loss_l)
        
        # save best one fold result
        if val_loss_l < single_best:
            single_best = val_loss_l
            single_best_pred = cache_test_pred
            print('new single best')
        
        cache_test_pred = np.zeros((153164,6))
        
        # avg k fold
        all_train_loss_l += train_loss_l/k
        all_val_loss_l += val_loss_l/k
        print('========================')
    test_pred = test_pred/k
    print('all train avg',all_train_loss_l,'all val avg',all_val_loss_l)
    return test_pred, single_best_pred

print('done')

done


In [3]:
# xgb_res,b = simple_ens('xgb',k=3)
# sample_submission = pd.read_csv("../input/sample_submission.csv")
# sample_submission[list_classes] = xgb_res
# sample_submission.to_csv("../results/xgb_ens_new_csv_fold3.gz", index=False, compression='gzip')
# print(sample_submission.head())
# print('save done')


In [4]:
%%time
xgb_res,b = simple_ens('xgb',k=5)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res
sample_submission.to_csv("../results/xgb_ens_new_csv_fold5.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')
# all train avg 0.031309680463 all val avg 0.0368863155994, PUB 9862
# fix lr, mnb bug
# all train avg 0.030348931425 all val avg 0.0361817765443

[0]	train-logloss:0.649111	valid-logloss:0.649119
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[535]	train-logloss:0.065225	valid-logloss:0.076604

0
0.0641850858985 0.0766447678144
[0]	train-logloss:0.645619	valid-logloss:0.645626
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[186]	train-logloss:0.017632	valid-logloss:0.019889

1
0.0167074980375 0.0199064503894
[0]	train-logloss:0.646797	valid-logloss:0.646762
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[189]	train-logloss:0.036592	valid-logloss:0.038148

2
0.0353862149427 0.0382073323206
[0]	train-logloss:0.644733	valid-logloss:0.644789
Multiple

[0]	train-logloss:0.64561	valid-logloss:0.645646
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[191]	train-logloss:0.017124	valid-logloss:0.021653

1
0.0162530579467 0.0217004639045
[0]	train-logloss:0.646746	valid-logloss:0.646842
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[228]	train-logloss:0.034974	valid-logloss:0.040641

2
0.0339700021855 0.040657616653
[0]	train-logloss:0.64473	valid-logloss:0.644786
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[188]	train-logloss:0.00442	valid-logloss:0.008169

3
0.00386511722787 0.00823849938319
[0]	train-logloss:0.647696	valid-logloss:0.647801
Multiple e

In [5]:
%%time
xgb_res,b = simple_ens('xgb',k=10)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res
sample_submission.to_csv("../results/xgb_ens_new_csv_fold10.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')
# all train avg 0.0317569652451 all val avg 0.0368260957171, PUB 9863
# fix lr, mnb bug
# all train avg 0.0306159230011 all val avg 0.036115284694,  PUB 9866

[0]	train-logloss:0.649065	valid-logloss:0.649084
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[554]	train-logloss:0.065993	valid-logloss:0.075443

0
0.0650905625654 0.0755177228479
[0]	train-logloss:0.645615	valid-logloss:0.645624
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[202]	train-logloss:0.017487	valid-logloss:0.020339

1
0.016650586868 0.0204082655098
[0]	train-logloss:0.646755	valid-logloss:0.646826
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[213]	train-logloss:0.035928	valid-logloss:0.040592

2
0.0348656060959 0.0406045137698
[0]	train-logloss:0.644742	valid-logloss:0.644802
Multiple 

[0]	train-logloss:0.645607	valid-logloss:0.645702
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[185]	train-logloss:0.017739	valid-logloss:0.021006

1
0.0169313970343 0.0210958680784
[0]	train-logloss:0.646787	valid-logloss:0.646699
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[264]	train-logloss:0.035147	valid-logloss:0.037304

2
0.0341264354975 0.0373493854267
[0]	train-logloss:0.644746	valid-logloss:0.644771
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[211]	train-logloss:0.004461	valid-logloss:0.007172

3
0.00390951516522 0.00726445102902
[0]	train-logloss:0.647697	valid-logloss:0.647816
Multip

Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[180]	train-logloss:0.036393	valid-logloss:0.042582

2
0.0354274963554 0.0426724953115
[0]	train-logloss:0.644736	valid-logloss:0.644805
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[205]	train-logloss:0.004395	valid-logloss:0.009095

3
0.00387228319283 0.00915784683019
[0]	train-logloss:0.647703	valid-logloss:0.647708
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[322]	train-logloss:0.048516	valid-logloss:0.05335

4
0.0475146604697 0.0533930975371
[0]	train-logloss:0.645352	valid-logloss:0.645362
Multiple eval metrics have been passed: 'valid-logloss' w